In [56]:
import pandas as pd
import numpy as np
import ast
import matplotlib.pyplot as plt
import os
import keras_tuner as kt
import wandb
import tqdm
import tensorflow as tf
import keras

from tensorflow.keras import layers, Model, optimizers, losses, callbacks
from tensorflow import GradientTape
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split, GridSearchCV
from keras.callbacks import EarlyStopping
from scikeras.wrappers import KerasClassifier

In [57]:
train = pd.read_csv('../data/preprocessed/train_data.csv')
test = pd.read_csv('../data/preprocessed/test_data.csv')
val = pd.read_csv('../data/preprocessed/val_data.csv')

In [58]:
train.shape, test.shape, val.shape

((50176, 48), (5469, 48), (10752, 48))

In [59]:
X_train = train.drop("Is high risk", axis=1)
Y_train = train["Is high risk"]

X_test = test.drop("Is high risk", axis=1)
Y_test = test["Is high risk"]

X_val = val.drop("Is high risk", axis=1)
Y_val = val["Is high risk"]

In [60]:
# wandb.login()

In [61]:
def Model():
    x = layers.Input(shape=(X_train.shape[1],))

    h = layers.Dense(64, activation='relu')(x)
    h = layers.Dense(64, activation='relu')(h)
    h = layers.Dense(64, activation='relu')(h)
    h = layers.Dense(64, activation='relu')(h)

    y = layers.Dense(1, activation='sigmoid')(h)

    model = keras.Model(inputs=x, outputs=y)
    # model.summary()

    return model

def train_step(x, y, model, optimizer, loss_fn, train_acc_metric):
    with GradientTape() as tape:
        logits = model(x, training=True)
        loss_value = loss_fn(y, logits)

    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))

    train_acc_metric.update_state(y, logits)

    return loss_value

def test_step(x, y, model, loss_fn, val_acc_metric):
    val_logits = model(x, training=False)
    loss_value = loss_fn(y, val_logits)
    val_acc_metric.update_state(y, val_logits)

    return loss_value

In [62]:
def train(train_dataset,
          val_dataset, 
          model,
          optimizer,
          loss_fn,
          train_acc_metric,
          val_acc_metric,
          epochs=1000, 
          log_step=200, 
          val_log_step=50):
    for epoch in range(epochs):
        print("\nStart of epoch %d" % (epoch,))

        train_loss = []   
        val_loss = []

        # Iterate over the batches of the dataset
        for step, (x_batch_train, y_batch_train) in tqdm.tqdm(enumerate(train_dataset), total=len(train_dataset)):
            loss_value = train_step(x_batch_train, y_batch_train, 
                                    model, optimizer, 
                                    loss_fn, train_acc_metric)
            train_loss.append(float(loss_value))

        # Run a validation loop at the end of each epoch
        for step, (x_batch_val, y_batch_val) in enumerate(val_dataset):
            val_loss_value = test_step(x_batch_val, y_batch_val, 
                                       model, loss_fn, 
                                       val_acc_metric)
            val_loss.append(float(val_loss_value))
            
        # Display metrics at the end of each epoch
        train_acc = train_acc_metric.result()
        print("Training acc over epoch: %.4f" % (float(train_acc),))

        val_acc = val_acc_metric.result()
        print("Validation acc: %.4f" % (float(val_acc),))

        # Reset metrics at the end of each epoch
        train_acc_metric.reset_state()
        val_acc_metric.reset_state()

        # 3️⃣ log metrics using wandb.log
        wandb.log({'epochs': epoch,
                   'loss': np.mean(train_loss),
                   'acc': float(train_acc), 
                   'val_loss': np.mean(val_loss),
                   'val_acc':float(val_acc)})

In [63]:
sweep_config = {
  'method': 'grid', 
  'metric': {
      'name': 'val_loss',
      'goal': 'minimize'
  },
  'early_terminate':{
      'type': 'hyperband',
      'min_iter': 5
  },
  'parameters': {
      'batch_size': {
          'values': [64, 128, 256]
      }
    #   'learning_rate':{
    #       'values': [0.01, 0.005, 0.001, 0.0005, 0.0001]
    #   }
  }
}

In [64]:
def sweep_train(config_defaults=None):
    # Set default values
    config_defaults = {
        "batch_size": 32,
        "learning_rate": 0.001
    }
    # Initialize wandb with a sample project name
    wandb.init(config=config_defaults)  # this gets over-written in the Sweep

    # Specify the other hyperparameters to the configuration, if any
    wandb.config.epochs = 2
    wandb.config.log_step = 20
    wandb.config.val_log_step = 50
    wandb.config.architecture_name = "test"
    wandb.config.dataset_name = "Credit Card"

    # build input pipeline using tf.data
    train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
    train_dataset = (train_dataset.batch(wandb.config.batch_size)
                                  .prefetch(buffer_size=tf.data.AUTOTUNE)
                                  .shuffle(buffer_size=50176)
                                  )

    val_dataset = tf.data.Dataset.from_tensor_slices((X_val, Y_val))
    val_dataset = (val_dataset.batch(wandb.config.batch_size)
                              .prefetch(buffer_size=tf.data.AUTOTUNE)
                              .shuffle(buffer_size=10752)
                              )

    # initialize model
    model = Model()

    # Instantiate an optimizer to train the model.
    # optimizer = keras.optimizers.SGD(learning_rate=wandb.config.learning_rate)
    optimizer = optimizers.Adam(learning_rate=wandb.config.learning_rate, 
                            beta_1=0.9, 
                            beta_2=0.999, 
                            amsgrad=True)
    # Instantiate a loss function.
    loss_fn = keras.losses.BinaryCrossentropy(from_logits=False)

    # Prepare the metrics.
    train_acc_metric = keras.metrics.BinaryAccuracy()
    val_acc_metric = keras.metrics.BinaryAccuracy()

    train(train_dataset,
          val_dataset, 
          model,
          optimizer,
          loss_fn,
          train_acc_metric,
          val_acc_metric,
          epochs=wandb.config.epochs, 
          log_step=wandb.config.log_step, 
          val_log_step=wandb.config.val_log_step)

In [65]:
sweep_id = wandb.sweep(sweep_config, project="test-sweep")

Create sweep with ID: k70tbv0d
Sweep URL: https://wandb.ai/lokrau/test-sweep/sweeps/k70tbv0d


In [66]:
wandb.agent(sweep_id, function=sweep_train, count=5)

wandb: Agent Starting Run: n9ir8630 with config:
wandb: 	batch_size: 64
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.



Start of epoch 0


100%|██████████| 784/784 [00:15<00:00, 49.13it/s]
2024-06-07 18:08:45.551819: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Training acc over epoch: 0.5102
Validation acc: 0.5058

Start of epoch 1


100%|██████████| 784/784 [00:15<00:00, 49.28it/s]
2024-06-07 18:09:01.937639: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


...

Training acc over epoch: 0.4782
Validation acc: 0.5020

Start of epoch 1


100%|██████████| 196/196 [00:04<00:00, 45.90it/s]
2024-06-07 18:09:54.795631: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Training acc over epoch: 0.4869
Validation acc: 0.5201


acc,▁█
epochs,▁█
loss,█▁
val_acc,▁█
val_loss,█▁
acc,0.48695
epochs,1
loss,291.68493
val_acc,0.52009
val_loss,75.64218


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


In [67]:
sweep_id

'k70tbv0d'

In [68]:
api = wandb.Api()
sweep_id = 'lokrau/test-sweep/sweeps/' + sweep_id

sweep = api.sweep(sweep_id)
runs = sweep.runs


In [69]:
wandb.finish()

In [70]:
results = []
for run in runs:
    run_data = {
        'batch_size': run.config.get('batch_size'),
        'val_accuracy': run.summary.get('val_acc'),
        'val_loss': run.summary.get('val_loss'),
        'accuracy': run.summary.get('acc'),
        'loss': run.summary.get('loss'),
    }
    results.append(run_data)

In [71]:
results_df = pd.DataFrame(results)
results_df

,batch_size,val_accuracy,val_loss,accuracy,loss
0,256,0.520089,75.642179,0.486946,291.684931
1,128,0.505580,118.574313,0.538604,203.276822
2,64,0.502883,3.446066,0.508092,11.182989


In [72]:
best_run = results_df.loc[results_df['val_loss'].idxmin()]
best_batch_size = best_run['batch_size']
best_val_loss = best_run['val_loss']

In [73]:
print(f'Best batch size: {best_batch_size} with validation accuracy: {best_val_loss}')

Best batch size: 64.0 with validation accuracy: 3.4460660093242215


In [74]:
model = Model()
model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 47)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 64)             │         3,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,617 (61.00 KB)

 Trainable params: 15,617 (61.00 KB)

 Non-trainable params: 0 (0.00 B)

In [78]:
early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    min_delta=0.001,     # Minimum change to qualify as an improvement
    patience=10,         # How many epochs to wait after last time val loss improved
    verbose=1,
    mode='min',          # The training will stop when the quantity monitored has stopped decreasing
    restore_best_weights=True  # Restores model weights from the epoch with the best value of the monitored quantity.
)

In [76]:
optimizer = optimizers.Adam(learning_rate=0.001, 
                            beta_1=0.9, 
                            beta_2=0.999, 
                            amsgrad=True)

model.compile(optimizer=optimizer, 
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [79]:
batch_size = int(best_batch_size)
epochs = 1000

history = model.fit(X_train, 
                    Y_train, 
                    batch_size=batch_size,
                    validation_data=(X_val, Y_val),
                    callbacks=[early_stopping],
                    epochs=epochs)

Epoch 1/1000
784/784 ━━━━━━━━━━━━━━━━━━━━ 2s 928us/step - accuracy: 0.5123 - loss: 257.0394 - val_accuracy: 0.5106 - val_loss: 30.3661
Epoch 2/1000
784/784 ━━━━━━━━━━━━━━━━━━━━ 1s 807us/step - accuracy: 0.5176 - loss: 48.8698 - val_accuracy: 0.5451 - val_loss: 6.3567
Epoch 3/1000
784/784 ━━━━━━━━━━━━━━━━━━━━ 1s 800us/step - accuracy: 0.5212 - loss: 15.6084 - val_accuracy: 0.5190 - val_loss: 11.7677
Epoch 4/1000
784/784 ━━━━━━━━━━━━━━━━━━━━ 1s 926us/step - accuracy: 0.5196 - loss: 10.9211 - val_accuracy: 0.4958 - val_loss: 8.2716
Epoch 5/1000
784/784 ━━━━━━━━━━━━━━━━━━━━ 1s 844us/step - accuracy: 0.5219 - loss: 6.7347 - val_accuracy: 0.5490 - val_loss: 1.9637
Epoch 6/1000
784/784 ━━━━━━━━━━━━━━━━━━━━ 1s 837us/step - accuracy: 0.5281 - loss: 4.5576 - val_accuracy: 0.5377 - val_loss: 1.9161
Epoch 7/1000
784/784 ━━━━━━━━━━━━━━━━━━━━ 1s 839us/step - accuracy: 0.5323 - loss: 2.9214 - val_accuracy: 0.5227 - val_loss: 1.4912
Epoch 8/1000
784/784 ━━━━━━━━━━━━━━━━━━━━ 1s 848us/step - accuracy: 0